# Installing Keycloak

## Prerequisites
- [Learning Jupyter](https://github.com/NephTek/Public_Notebooks/blob/main/getting-started/learning_jupyter.ipynb)
- Install Jupyter bash kernel:
  - [Windows](https://github.com/NephTek/Public_Notebooks/blob/main/getting-started/windows/install_jupyter_bash_kernel.ipynb)
  - [Mac](https://github.com/NephTek/Public_Notebooks/blob/main/getting-started/mac/install_jupyter_bash_kernel.ipynb)
  - [Linux](https://github.com/NephTek/Public_Notebooks/blob/main/getting-started/linux/install_jupyter_bash_kernel.ipynb)
- [Install Kubernetes](https://github.com/NephTek/Public_Notebooks/blob/main/kubernetes/install_k8s_cluster.ipynb)
- [Install cert-manager](https://github.com/NephTek/Public_Notebooks/blob/main/cert-manager/install_cert-manager.ipynb)
- [Install Monitoring](https://github.com/NephTek/Public_Notebooks/blob/main/monitoring/install_kube-prometheus-stack.ipynb)
- [Install Logging](https://github.com/NephTek/Public_Notebooks/blob/main/logging/install_loki.ipynb)
- [Install Ingress](https://github.com/NephTek/Public_Notebooks/blob/main/ingress/install_ingress-nginx.ipynb)
- [Install Postgres](https://github.com/NephTek/Public_Notebooks/blob/main/postgres/install_postgres.ipynb)

## Start Your K8s Cluster
Start by ensuring your cluster is available and running.

In [1]:
kubectl get nodes

NAME                   STATUS   ROLES                          AGE   VERSION
lima-rancher-desktop   Ready    builder,control-plane,master   19d   v1.23.3+k3s1


In [2]:
ls

Installing Keycloak.ipynb	bashkernelinstall.ipynb
LICENSE				k3d_getting_started.ipynb
README.md			keycloak-operator


## Install Postgres
Keycloak is an Identity Provider.  Keycloak requires a database to store its data to manage identity.  While the Keycloak operator supports an internally managed Postgres instance, the instance is intended only for development purposes only.  Instead, we're going to use an external instance of Postgres created using the Zalando Postgres Operator.

In [1]:
git clone https://github.com/keycloak/keycloak-operator.git

Cloning into 'keycloak-operator'...
remote: Enumerating objects: 12617, done.
remote: Counting objects: 100% (2372/2372), done.
remote: Compressing objects: 100% (407/407), done.
remote: Total 12617 (delta 2112), reused 1982 (delta 1961), pack-reused 10245
Receiving objects: 100% (12617/12617), 11.41 MiB | 1.94 MiB/s, done.
Resolving deltas: 100% (7218/7218), done.


: 1

In [1]:
cd keycloak-operator

In [2]:
make cluster/prepare

customresourcedefinition.apiextensions.k8s.io/keycloakbackups.keycloak.org created
customresourcedefinition.apiextensions.k8s.io/keycloakbackups.keycloak.org unchanged
customresourcedefinition.apiextensions.k8s.io/keycloakclients.keycloak.org created
customresourcedefinition.apiextensions.k8s.io/keycloakclients.keycloak.org unchanged
customresourcedefinition.apiextensions.k8s.io/keycloakrealms.keycloak.org created
customresourcedefinition.apiextensions.k8s.io/keycloakrealms.keycloak.org unchanged
customresourcedefinition.apiextensions.k8s.io/keycloaks.keycloak.org created
customresourcedefinition.apiextensions.k8s.io/keycloaks.keycloak.org unchanged
customresourcedefinition.apiextensions.k8s.io/keycloakusers.keycloak.org created
customresourcedefinition.apiextensions.k8s.io/keycloakusers.keycloak.org unchanged
namespace/keycloak created
role.rbac.authorization.k8s.io/keycloak-operator created
rolebinding.rbac.authorization.k8s.io/keycloak-operator created
serviceaccount/keycloak-operat

In [3]:
kubectl apply -f deploy/operator.yaml -n keycloak

deployment.apps/keycloak-operator created


In [4]:
cd ..

In [5]:
kubectl apply -f keycloak-postgres.yaml

postgresql.acid.zalan.do/acid-keycloak-cluster created


In [2]:
PGPASSWORD=$(kubectl get secret keycloak.acid-keycloak-cluster.credentials.postgresql.acid.zalan.do -n keycloak -o 'jsonpath={.data.password}' | base64 -d)

In [3]:
kubectl create secret generic keycloak-db-secret -n keycloak \
        --from-literal=POSTGRES_DATABASE=keycloak \
        --from-literal=POSTGRES_EXTERNAL_ADDRESS=acid-keycloak-cluster.keycloak.svc.cluster.local \
        --from-literal=POSTGRES_EXTERNAL_PORT=5432 \
        --from-literal=POSTGRES_USERNAME=keycloak \
        --from-literal=POSTGRES_PASSWORD=$PGPASSWORD

secret/keycloak-db-secret created


In [4]:
kubectl label secret keycloak-db-secret app=keycloak

secret/keycloak-db-secret labeled


In [15]:
kubectl apply -f keycloak-sso.yaml

keycloak.keycloak.org/keycloak-sso created


In [16]:
kubectl get pods -n keycloak

NAME                                 READY   STATUS    RESTARTS        AGE
keycloak-operator-846f9d566f-xzhlm   1/1     Running   4 (4h17m ago)   21d
acid-keycloak-cluster-0              1/1     Running   0               16m
keycloak-0                           0/1     Running   0               7s


In [5]:
kubectl apply -f keycloak-ingressroutetcp.yaml

ingressroutetcp.traefik.containo.us/keycloak unchanged


### Finished

### Next Steps